In [ ]:
import os
import cv2
import numpy as np
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle
from skimage import img_as_ubyte, img_as_float
from scipy.ndimage import filters

# Guided Filter function
def guided_filter(image, radius=2, epsilon=0.1):
    guide = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    guide = img_as_float(guide)
    p = img_as_float(image)
    q = denoise_bilateral(guide, p, sigma_spatial=radius, sigma_color=epsilon)
    return q

# Bilateral Filter with Edge Weights function
def bilateral_filter_with_edge_weights(image, d=9, sigma_color=75, sigma_space=75):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    dilated_edges = cv2.dilate(edges, None, iterations=2)
    smoothed_image = denoise_bilateral(image, sigma_color=sigma_color, sigma_spatial=sigma_space, multichannel=True)
    smoothed_image[dilated_edges > 0] = image[dilated_edges > 0]
    return smoothed_image

# Function to apply edge-aware smoothing and save the result to a folder
def apply_edge_smoothing(original_folder, output_folder, smoothing_function, **params):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(original_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(original_folder, filename)
            original_image = cv2.imread(image_path)

            # Apply the edge-aware smoothing technique
            smoothed_image = smoothing_function(original_image, **params)

            # Convert the smoothed image back to uint8 format
            smoothed_image = img_as_ubyte(smoothed_image)

            # Save the smoothed image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, smoothed_image)

# Define the paths for the original segmented images folder and output folders for each technique
original_images_folder = "path/to/original_images_folder"
output_guided_filter_folder = "path/to/output/guided_filter"
output_bilateral_with_edge_weights_folder = "path/to/output/bilateral_with_edge_weights"

# Apply edge-aware smoothing techniques to the original images
apply_edge_smoothing(original_images_folder, output_guided_filter_folder, guided_filter, radius=2, epsilon=0.1)
apply_edge_smoothing(original_images_folder, output_bilateral_with_edge_weights_folder, bilateral_filter_with_edge_weights, d=9, sigma_color=75, sigma_space=75)
